#  Assignment_5-Ragul M,1P24CS014

In [45]:
from ISLP import load_data
import pandas as pd
import numpy as np
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.linear_model import LinearRegression

In [46]:
np.random.seed(0)
x = np.random.normal(size=100)
eps = np.random.normal(size=100)

In [47]:
beta0,beta1,beta2,beta3 =1,2,3,4
y = beta0 + beta1*x + beta2*x**2 + beta3*x**3 + eps

In [48]:
df = pd.DataFrame({"X": x})
for i in range(2, 11):
    df[f"X{i}"] = x**i
df.head()

,X,X2,X3,X4,X5,X6,X7,X8,X9,X10
0,1.764052,3.111881,5.489520,9.683801,17.082733,30.134834,53.159425,93.776009,165.425788,291.819750
1,0.400157,0.160126,0.064075,0.025640,0.010260,0.004106,0.001643,0.000657,0.000263,0.000105
2,0.978738,0.957928,0.937561,0.917626,0.898116,0.879020,0.860330,0.842038,0.824134,0.806612
3,2.240893,5.021602,11.252875,25.216490,56.507461,126.627185,283.757997,635.871366,1424.919820,3193.093133
4,1.867558,3.487773,6.513618,12.164559,22.718020,42.427220,79.235294,147.976506,276.354706,516.108440


In [49]:
lr = LinearRegression()

sfs_forward = SFS(lr,
                  k_features="best",
                  forward=True,
                  floating=False,
                  scoring="r2",
                  cv=5)
sfs_forward = sfs_forward.fit(df.values, y)
print("froward selection:", sfs_forward.k_feature_names_)

froward selection: ('0', '1', '2', '3', '4', '6')


In [50]:
lr = LinearRegression()


sfs_backward = SFS(lr,
                   k_features="best",
                   forward=False,
                   floating=False,
                   scoring="r2",
                   cv=5)

sfs_backward = sfs_backward.fit(df.values, y)
print("backward selection:", sfs_backward.k_feature_names_)


backward selection: ('0', '2', '3', '4', '5', '6', '7', '8', '9')


In [51]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso

In [52]:
alphas = np.logspace(-3, 3, 100)

lasso_cv_scores = []
for a in alphas:
    lasso = Lasso(alpha=a,max_iter=5000)
    scores = cross_val_score(lasso, df, y, cv=5, scoring='neg_mean_squared_error')
    lasso_cv_scores.append(np.mean(scores))
best_alpha_lasso = alphas[np.argmin(lasso_cv_scores)]
lasso_best = Lasso(alpha=best_alpha_lasso, max_iter=5000).fit(df, y)

print(f"Best Lasso α (λ): {best_alpha_lasso}")
print("Lasso Coefficients (many may be 0):", lasso_best.coef_)

C:\Users\ragul\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.327e+01, tolerance: 1.658e+00
  model = cd_fast.enet_coordinate_descent(
C:\Users\ragul\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.040e+01, tolerance: 1.472e+00
  model = cd_fast.enet_coordinate_descent(
C:\Users\ragul\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider in

Best Lasso α (λ): 46.41588833612782
Lasso Coefficients (many may be 0): [ 0.          0.          0.          0.          0.          0.
  0.17876054  0.         -0.          0.00586049]


UNIT 5

import packages

In [53]:
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeCV, LassoCV, LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
from ISLP  import load_data
import numpy as np



#### load data

In [54]:
clg = load_data("college")
clg

,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
0,Yes,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
1,Yes,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
2,Yes,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
3,Yes,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
4,Yes,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772,No,2197,1515,543,4,26,3089,2029,6797,3900,500,1200,60,60,21.0,14,4469,40
773,Yes,1959,1805,695,24,47,2849,1107,11520,4960,600,1250,73,75,13.3,31,9189,83
774,Yes,2097,1915,695,34,61,2793,166,6900,4200,617,781,67,75,14.4,20,8323,49
775,Yes,10705,2453,1317,95,99,5217,83,19840,6510,630,2115,96,96,5.8,49,40386,99


1.A

In [55]:
clg['Private'] = (clg['Private']=='Yes').astype(int)
clg

,Private,Apps,Accept,Enroll,Top10perc,Top25perc,F.Undergrad,P.Undergrad,Outstate,Room.Board,Books,Personal,PhD,Terminal,S.F.Ratio,perc.alumni,Expend,Grad.Rate
0,1,1660,1232,721,23,52,2885,537,7440,3300,450,2200,70,78,18.1,12,7041,60
1,1,2186,1924,512,16,29,2683,1227,12280,6450,750,1500,29,30,12.2,16,10527,56
2,1,1428,1097,336,22,50,1036,99,11250,3750,400,1165,53,66,12.9,30,8735,54
3,1,417,349,137,60,89,510,63,12960,5450,450,875,92,97,7.7,37,19016,59
4,1,193,146,55,16,44,249,869,7560,4120,800,1500,76,72,11.9,2,10922,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772,0,2197,1515,543,4,26,3089,2029,6797,3900,500,1200,60,60,21.0,14,4469,40
773,1,1959,1805,695,24,47,2849,1107,11520,4960,600,1250,73,75,13.3,31,9189,83
774,1,2097,1915,695,34,61,2793,166,6900,4200,617,781,67,75,14.4,20,8323,49
775,1,10705,2453,1317,95,99,5217,83,19840,6510,630,2115,96,96,5.8,49,40386,99


In [56]:
x = clg.drop(columns=["Apps"])
y = clg["Apps"]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1)
x_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 543 entries, 214 to 37
Data columns (total 17 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Private      543 non-null    int32  
 1   Accept       543 non-null    int64  
 2   Enroll       543 non-null    int64  
 3   Top10perc    543 non-null    int64  
 4   Top25perc    543 non-null    int64  
 5   F.Undergrad  543 non-null    int64  
 6   P.Undergrad  543 non-null    int64  
 7   Outstate     543 non-null    int64  
 8   Room.Board   543 non-null    int64  
 9   Books        543 non-null    int64  
 10  Personal     543 non-null    int64  
 11  PhD          543 non-null    int64  
 12  Terminal     543 non-null    int64  
 13  S.F.Ratio    543 non-null    float64
 14  perc.alumni  543 non-null    int64  
 15  Expend       543 non-null    int64  
 16  Grad.Rate    543 non-null    int64  
dtypes: float64(1), int32(1), int64(15)
memory usage: 74.2 KB


Interpretation OF A

converting the 'Private' column to binary (1 for 'Yes', 0 for 'No') and preparing data for modeling by separating predictors (x) and target (y). Then you're splitting the data into training and test sets, and checking the structure of the training predictors.


1.B

In [57]:
x_train=sm.add_constant(x_train)
x_test=sm.add_constant(x_test)
model = sm.OLS(y_train, x_train)
results = model.fit()
pred = results.predict(x_test)
np.mean((y_test - pred)**2)
mse=mean_squared_error(y_test, pred)
mse
rmse=np.sqrt(mse)
rmse

801.7193384553052

Interpretation OF B

built a linear regression model to predict college application numbers using multiple features. After fitting the model and generating predictions, you evaluated its accuracy using Mean Squared Error (MSE) and Root Mean Squared Error (RMSE). A lower RMSE indicates better predictive performance and closer alignment with actual application counts.

1.C

In [58]:
alphas = np.logspace(-3, 3, 100)
ridge_cv = RidgeCV(alphas=alphas)
ridge_cv.fit(x_train, y_train)
best_alpha_ridge = ridge_cv.alpha_
print(f"Best Ridge α (λ): {best_alpha_ridge}")
y_pred_ridge = ridge_cv.predict(x_test)
ridge_test_mse = mean_squared_error(y_test, y_pred_ridge)
ridge_test_rmse = np.sqrt(ridge_test_mse)
ridge_test_mse
ridge_test_rmse


Best Ridge α (λ): 40.37017258596558


811.7113568302875

Interpretation OF C

 Ridge regression model with cross-validation to find the best regularization strength (α), which helps prevent overfitting. After fitting the model, you predicted test values and calculated the Mean Squared Error (MSE) and Root Mean Squared Error (RMSE) to assess prediction accuracy. A lower RMSE suggests the Ridge model generalizes well to unseen data.

1.D

In [59]:

alphas = np.logspace(-3, 3, 100)
lasso_cv = LassoCV(alphas=alphas, max_iter=5000, random_state=42)
lasso_cv.fit(x_train, y_train)
best_alpha_lasso = lasso_cv.alpha_
print(f"Best lasso α (λ): {best_alpha_lasso}")
y_pred_lasso = lasso_cv.predict(x_test)
lasso_test_mse = mean_squared_error(y_test, y_pred_lasso)
lasso_test_rmse = np.sqrt(lasso_test_mse)
lasso_test_mse
lasso_test_rmse

Best lasso α (λ): 187.3817422860383


823.8258882661901

Interpretation OF D

Lasso regression model with cross-validation to find the optimal regularization strength (α), which helps reduce overfitting and select important predictors. After fitting, you predicted application counts and evaluated performance using MSE and RMSE. A lower RMSE indicates better predictive accuracy and model generalization.

1.E

In [60]:

pca = PCA(n_components=5)
x_train_pca = pca.fit_transform(x_train)
x_test_pca = pca.transform(x_test)

pcr = LinearRegression()
pcr.fit(x_train_pca, y_train)
y_pred_pcr = pcr.predict(x_test_pca)
mse_pcr = mean_squared_error(y_test, y_pred_pcr)
rmse_pcr = np.sqrt(mse_pcr) 
mse_pcr
rmse_pcr


958.0846684422346

Interpretation OF E

Principal Component Regression (PCR) by reducing your predictors to 5 principal components before fitting a linear model. This helps handle multicollinearity and reduce dimensionality. The resulting RMSE shows how well the PCR model predicts college application numbers on unseen data.

1.F

In [61]:
pls = PLSRegression(n_components=5)
x_train_pls = pls.fit_transform(x_train, y_train)[0]
x_test_pls = pls.transform(x_test)

lr = LinearRegression()
lr.fit(x_train_pls, y_train)
y_pred_pls = lr.predict(x_test_pls)
mse_pls = mean_squared_error(y_test, y_pred_pls)
rmse_pls = np.sqrt(mse_pls)
mse_pls
rmse_pls

892.3597520170177

Interpretation OF F

Partial Least Squares (PLS) regression using 5 components to capture key patterns between predictors and the target. After transforming the data and fitting a linear model, you evaluated prediction accuracy using MSE and RMSE. A lower RMSE indicates that the PLS model effectively balances dimensionality reduction with predictive performance.

1.G

Answer

A variety of regression techniques—OLS, Ridge, Lasso, PCR, and PLS—can all predict college application numbers with reasonable accuracy, as indicated by similar RMSE values across models. This suggests that no single method dramatically outperforms the others in terms of predictive performance.

**Key strengths of each method:**
- **OLS**: Serves as a baseline. If its RMSE is close to regularized or reduced models, it may be sufficient.
- **Ridge & Lasso**: Help control overfitting. Lasso also performs variable selection by shrinking some coefficients to zero.
- **PCR & PLS**: Reduce dimensionality, which is especially useful when predictors are highly correlated.

**Summary:**  
Choose OLS for simplicity if performance is comparable. Use Ridge or Lasso for regularization and variable selection. Opt for PCR or PLS when dealing with multicollinearity or high-dimensional data.
